In [13]:
#! /usr/bin/env python3

# data libraries
import sys
import os
import datetime as dt  # Python standard library datetime  module
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
from netCDF4 import Dataset,num2date  # http://code.google.com/p/netcdf4-python/
import pandas as pd


# plotting libraries
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.style as style
from matplotlib import cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point 
import cartopy.mpl.ticker as cticker
import glob

np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_rows = 4000

## Defining functions for Analysis

### iceshelf_stats(path,path2) = f1,model_names,ym,modeltemps,var_era5_spatial_avg,ym_era5,model_temps_cropped,shelf_name,gridsize

#### Inputs
- path: is the path for the CMIP6 model data for the iceshelf. This path contains the 32 models nc files cropped to the specific object. You can choose to apply the path for seasonal (Austral Summer- December, January, and February), yearly, or monthly model data for each shelf.
- path2:is the path for the ERA5 reanalysis data over the iceshelf. You can choose to apply the path for seasonal (Austral Summer- December, January, and February), yearly, or monthly reanalysis data for each shelf.

#### Outputs
- f1: a line plot of all 32 CMIP6 model data overlayed with the mean of the CMIP6 model data and the ERA5 reanalysis data
- model_names: a list of the CMIP6 models used in the analysis pulled from the file path name
- ym: is an array of years and months from the model data
- modeltemps: a list of each CMIP6 models temperature over the region across the years stored in ym
- var_era5_spatial_avg: ERA5 list of temperature over the region across ERA5's timescale (years stored in ym_era5)
- ym_era5: an array of years and months from the ERA5 data
- model_temps_cropped: the model temps cropped to the timescale of the ERA5 data
- shelf_name: name of shelf that you are analyzing for each dataset
- gridsize: size of the area for the shelf you are averaging over.

### bias_analysis(model_temps_cropped,era5_temps) = f2, df_sortted

A function that takes in the temperature from both the models and the ERA5 data and produces a plot with a bias analysis and dataframe sorted by bias. 

#### Inputs
- model_temps_cropped: the model temps cropped to the timescale of the ERA5 data
- era5_temps: ERA5 list of temperature over the region across ERA5's timescale (years stored in ym_era5)

#### Outputs
- f2: bar plot of bias for each CMIP6 model and the mean of the models. This bias is calculated by subtracting the ERA5 temperatures from the model temperatures across the timescale. The bias for eachmodel is produced by taking the mean of that variation. The mean of the CMIP6 models is calculated by taking the mean of the temperature at each time step and then performing the variation as mentioned above. 
- df_sortted: the dataframe of the CMIP6 models and their overall bias assigned based off of the ERA5 data in a dataframe sortted based on bias from negative to positive bias. The smallest bias will be towards the middle of the dataframe. 


In [14]:
def iceshelf_stats(path,path2): 
    
    # Lists of data pulled from paths
    fpath = list() # The path to location in my raid directory
    model_names = list() # The model name pulled from the file name of the data 
    modeltemps = list() # The temperature of the shelf for each year from model
    mean_annual_avg_temp = list() #mean of annual temperature values over historical period
    shelf_name = list()
    
    #List for bias analysis
    model_temps_cropped = list()
    
    # Plotting all of the models on one figure
    f1 = plt.figure(1,figsize=(15,12))
    for path in glob.iglob(f'{path}/*.nc'): #iterating through files in path specified
        
        #Reading in Datafiles
        fpath.append(path)
        fname = path
        datain = Dataset(fname, 'r')
        
        ##Splitting The Path to label the models
        head_tail = os.path.split(path) #makes the path a variable
        mystring = str(head_tail[1]) #turns the variable into a string
        x = mystring.split(".", 5)#parsing the model name by periods
        model_names.append(x[1]) # the second section of string is the model name
        shelf_name.appned(x[4])
        
        #Creating Variables
        var = datain.variables['tas'] #2meter surface temperture
        gridsize = var[1].size
        lat = datain.variables['latitude'][:] #latitude
        lon = datain.variables['longitude'][:] #longitude
        timevar = datain.variables['time'] #in days since 1850
        dtime = num2date(timevar[:],timevar.units) # The next two lines make the time variable readable
        ym = np.zeros((len(dtime),2), int)
        
        ##Spatial Average over sheet
        var_spatial_avg = np.mean(var[:,:,:],axis = (1,2)) #Taking the spatial average of yearly temperature over the iceshelf
        var_spatial_avg = var_spatial_avg - 273.15 #converting the temperature from Kelvin to Celsius
        modeltemps.append(var_spatial_avg[0:249]) #appending the model temps to a list
        model_temps_cropped.append(var_spatial_avg[110:173]) #appending the model temps to a list
        
        ##Time Component
        for i in range(len(dtime)): #Creating an aray of years from the time variable
            ym[i,0] = dtime[i].year
            ym[i,1] = dtime[i].month
            
        #Plotting the Data
        plt.figure(1)
        plt.plot(ym[:,0],var_spatial_avg,linewidth=2.5,label = '_nolegend_', color= 'lightgray')
        plt.xlabel("Time (Years)", fontsize = 18)
        plt.ylabel(r"Austral Summer Surface Temperature ($\degree$ C)", fontsize = 18)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.xlim(1959,2100)
        #plt.legend()
    
    #Plotting the ERA5 data on the same figure
    fname_era5 = path2
    datain_era5 = Dataset(fname_era5, 'r')


    var_era5 = datain_era5.variables['t2m'] #2meter surface temperture
    lat_era5 = datain_era5.variables['latitude'][:] #latitude
    lon_era5 = datain_era5.variables['longitude'][:] #longitude
    timevar_era5 = datain_era5.variables['time'] #in days since 1959
    dtime_era5 = num2date(timevar_era5[:],timevar_era5.units) # The next two lines make the time variable readable
    ym_era5 = np.zeros((len(dtime_era5),2), int)

    var_era5_spatial_avg = np.mean(var_era5[:,:,:],axis = (1,2)) #Taking the spatial average of yearly temperature over the iceshelf
    var_era5_spatial_avg = var_era5_spatial_avg - 273.15
    
    for i in range(len(dtime_era5)): #Creating an aray of years from the time variable
        ym_era5[i,0] = dtime_era5[i].year
    
    plt.figure(1)
    plt.plot(ym_era5[:,0],var_era5_spatial_avg,label = 'ERA5', color= 'blue')
    
    #Adding Mean Line to Plot
    models_mean = np.mean(modeltemps,axis=0)
    model_names.append('Model Mean')
    model_temps_cropped.append(models_mean[110:173])
    
    #Calculating Regression Line
    regress = linregress(ym[0:249,0], models_mean)
    regression_line = ym[0:249,0]*regress[0] + regress[1]

    plt.figure(1)
    plt.plot(ym[0:249,0],models_mean,label = 'Model Mean', color= 'orange')
    plt.legend()
    plt.close()
    
    return f1,model_names,ym,modeltemps,var_era5_spatial_avg,ym_era5,model_temps_cropped,shelf_name,gridsize
        
    

In [15]:
def bias_analysis(model_temps_cropped,era5_temps):
    #Bias Analysis

    variation = model_temps_cropped - era5_temps
    mean_bias = np.mean(variation,axis=1)

    #Making data frame for bias analysis
    df = pd.DataFrame(list(zip(shelf_mn,mean_bias)),columns=['Model Names','Mean Bias'])

    df_sortted = df.sort_values("Mean Bias",ascending=False)

    x = df_sortted['Model Names']
    y = df_sortted['Mean Bias']

    cmap = plt.cm.seismic
    norm = cm.colors.TwoSlopeNorm(vcenter=0,vmin=np.min(y),vmax=np.max(y))

    f2 = fig, ax = plt.subplots(figsize=(15,12))
    bars = ax.barh(x,y,align='center',color=cmap(norm(y.values)))
    ax.bar_label(bars,fmt = '%1.2f',padding=2,label_type='edge',fontsize=14)
    ax.set_xlim(left=min(y)-1)
    ax.set_xlabel(r"Mean Seasonal Temperature Bias($\degree$ C)", fontsize = 18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    return f2, df_sortted



## Creating Lists of file paths for the iceshelves and each CMIP6 model

In [16]:
rootdir = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/'

fpath_seasonal_shelves = list() # The path to location in my raid directory
fpath_yearly_shelves = list()
fpath_monthly_shelves = list()

fpath_reanalysis_seasonal_shelves = list() # The path to location in my raid directory
fpath_reanalysis_yearly_shelves = list()
fpath_reanalysis_monthly_shelves = list()

for subdir, dirs, files in os.walk(rootdir):
    for d in dirs:
        #os.path.join(subdir, d)
        path = os.path.join(subdir, d)
        head_tail = os.path.split(path)
        if head_tail[1] == 'seasonal' : 
            fpath_seasonal_shelves.append(path)
        elif head_tail[1] == 'yearly': 
            fpath_yearly_shelves.append(path)
        elif head_tail[1] == 'monthly': 
            fpath_monthly_shelves.append(path)

In [17]:
rootdir2 = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/seasavg'

fpath_seasonal_shelves_reanalysis = list() # The path to location in my raid directory
for subdir, dirs, files in os.walk(rootdir2):
    for file in files:
        path2 = os.path.join(subdir, file)
        head_tail2 = os.path.split(path2)
        fpath_seasonal_shelves_reanalysis.append(path2)


## Creating List of Reanalysis file paths for monthly, yearly and seasonal data filtered for each ice shelf


In [18]:
rootdir3 = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/yearavg'

fpath_yearly_shelves_reanalysis = list() # The path to location in my raid directory
for subdir, dirs, files in os.walk(rootdir3):
    for file in files:
        path3 = os.path.join(subdir, file)
        head_tail2 = os.path.split(path3)
        fpath_yearly_shelves_reanalysis.append(path3)


In [19]:
rootdir4 = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly'

fpath_monthly_shelves_reanalysis = list() # The path to location in my raid directory
for subdir, dirs, files in os.walk(rootdir4):
    for file in files:
        path4 = os.path.join(subdir, file)
        head_tail2 = os.path.split(path4)
        fpath_monthly_shelves_reanalysis.append(path4)

## Creating Dataframes of alphabetically sorted paths by iceshelf

In [28]:
#seasonal reanalysis dataframe with alphebatized shelf paths
reanalysis_seasonal = pd.DataFrame(list(zip(fpath_seasonal_shelves_reanalysis)),columns=['Shelf Path'])
reanalysis_seasonal_sorted = reanalysis_seasonal.sort_values("Shelf Path")

In [27]:
#seasonal reanalysis dataframe with alphebatized shelf paths
reanalysis_yearly = pd.DataFrame(list(zip(fpath_yearly_shelves_reanalysis)),columns=['Shelf Path'])
reanalysis_yearly_sorted = reanalysis_yearly.sort_values("Shelf Path")

In [22]:
#seasonal reanalysis dataframe with alphebatized shelf paths
reanalysis_monthly = pd.DataFrame(list(zip(fpath_monthly_shelves_reanalysis)),columns=['Shelf Path'])
reanalysis_monthly_sorted = reanalysis_monthly.sort_values('Shelf Path')

In [23]:
#seasonal model dataframe with alphebatized shelf paths
seasonal = pd.DataFrame(list(zip(fpath_seasonal_shelves)),columns=['Shelf Path'])
seasonal_sortted = seasonal.sort_values("Shelf Path")
print(seasonal_sortted['Shelf Path'][2])

#yearly model dataframe with alphebatized shelf paths
yearly = pd.DataFrame(list(zip(fpath_yearly_shelves)),columns=['Shelf Path'])
yearly_sortted = yearly.sort_values("Shelf Path")
print(yearly_sortted['Shelf Path'][2])

monthly = pd.DataFrame(list(zip(fpath_monthly_shelves)),columns=['Shelf Path'])
monthly_sortted = monthly.sort_values("Shelf Path")
print(monthly_sortted.values.tolist())


/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/atka/seasonal
/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/atka/yearly
[['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/abbot/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/amery/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/atka/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/bach/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/baudouin/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/borchgrevink/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/bruntStancomb/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/congerGlenzer/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/cook/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/cosgrove/monthly'], ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/crosson/monthly'], ['/raid01/mafields/ta

### Creating the model plots using iceshelf_stats

#### Monthly

In [29]:
### Creating List: 
shelf_model_names = list()
shelf_timescale = list()
shelf_model_temps = list()
shelf_era5_temps = list()
shelf_era5_timescale = list()
shelf_model_temps_over_era5_tiemscale = list()
shelf_names = list()
shelf_total_gridsize = list()

for path, path2 in zip(yearly_sortted.values.tolist(), reanalysis_yearly_sorted.values.tolist()):
    print(path,path2)
    shelf_var_yearly,shelf_mn,shelf_time,temps,era5_temps,era5_time,model_temps_cropped2era5_time,shelf_name,shelf_gridsize = iceshelf_stats(path,path2)
    model_names.append(shelf_mn)
    shelf_timescale.append(shelf_time)
    shelf_model_temps.append(temps)
    shelf_era5_temps.append(era5_temps)
    shelf_era5_timescale.append(era5_time)
    shelf_model_temps_over_era5_tiemscale.append(model_temps_cropped2era5_time)
    shelf_names.append(shelf_name)
    shelf_total_gridsize.append(shelf_gridsize)
    print('added!')
    
    

['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/abbot/yearly'] ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/yearavg/R1.ERA5.yearavg.abbot.nc']


FileNotFoundError: [Errno 2] No such file or directory: b"['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/yearavg/R1.ERA5.yearavg.abbot.nc']"

<Figure size 1080x864 with 0 Axes>

In [25]:
reanalysis_monthly_sorted.values.tolist()

[['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis.abbot.nc'],
 ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis.amery.nc'],
 ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis.atka.nc'],
 ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis.bach.nc'],
 ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis.baudouin.nc'],
 ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis.borchgrevink.nc'],
 ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis.bruntStancomb.nc'],
 ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis.congerGlenzer.nc'],
 ['/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis.cook.nc'],
 ['/raid01/mafields/tas/MODELS_f